In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from timm import create_model
from tqdm import tqdm
from PIL import Image
import os
from pathlib import Path
import urllib.request
import zipfile
import shutil
import random
import numpy as np

In [ ]:
# 랜덤 시드 고정
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"Random seed set to {seed}")

set_seed(42)

Random seed set to 42


In [ ]:
# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# PACS 데이터셋 자동 다운로드 (Kaggle Hub 사용)
def download_pacs():
    """Kaggle Hub를 이용한 PACS 데이터셋 다운로드"""
    try:
        import kagglehub
        print("Downloading PACS dataset from Kaggle...")
        path = kagglehub.dataset_download("nickfratto/pacs-dataset")
        print(f"Path to dataset files: {path}")
        return Path(path)
    except ImportError:
        print("kagglehub not installed. Installing...")
        os.system("pip install kagglehub")
        import kagglehub
        path = kagglehub.dataset_download("nickfratto/pacs-dataset")
        print(f"Path to dataset files: {path}")
        return Path(path)
    except Exception as e:
        print(f"Error downloading PACS: {e}")
        print("\nPlease install kagglehub and try again:")
        print("pip install kagglehub")
        raise

In [ ]:
# PACS 데이터셋 클래스
class PACSDataset(Dataset):
    def __init__(self, root_dir, domains, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.samples = []

        self.classes = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

        # ✅ 도메인 → 정수 ID 매핑 추가
        self.domain_to_idx = {
            'photo': 0,
            'art_painting': 1,
            'cartoon': 2,
            'sketch': 3
        }

        # 데이터 로드
        for domain in domains:
            possible_paths = [
                self.root_dir / domain,
                self.root_dir / 'PACS' / domain,
                self.root_dir / 'kfold' / domain,
                self.root_dir / 'pacs' / domain,
                self.root_dir / 'dct2_images' / 'dct2_images' / domain,
                self.root_dir / 'train' / domain,
                self.root_dir / 'val' / domain,
            ]

            domain_path = None
            for path in possible_paths:
                if path.exists():
                    domain_path = path
                    print(f"Found domain '{domain}' at: {path}")
                    break

            if domain_path is None:
                print(f"Warning: Domain {domain} not found in any expected location")
                continue

            for class_name in self.classes:
                class_path = domain_path / class_name
                if class_path.exists():
                    for ext in ['*.jpg', '*.png', '*.jpeg', '*.JPEG', '*.JPG']:
                        for img_file in class_path.glob(ext):
                            domain_id = self.domain_to_idx[domain]
                            self.samples.append(
                                (str(img_file), self.class_to_idx[class_name], domain_id)
                            )

        if len(self.samples) == 0:
            print(f"Warning: No samples found for domains: {domains}")
        else:
            print(f"Loaded {len(self.samples)} samples from domains: {domains}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label, domain_id = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        # ✅ domain_id까지 반환
        return image, label, domain_id

In [ ]:
class MobileNetV4WithFeature(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone

    def forward(self, x):
        # timm 내부 로직 그대로 사용
        feat_map = self.backbone.forward_features(x)
        logits = self.backbone.forward_head(feat_map)

        # 우리가 쓸 feature는 따로 뽑는다
        feat = feat_map.mean(dim=(2, 3))               # (B, C)

        return logits, feat


In [ ]:
# 하이퍼파라미터
batch_size = 32
num_epochs = 30
learning_rate = 0.0001
num_classes = 7
weight_decay = 0.0005

In [ ]:
# PACS 도메인
all_domains = ['photo', 'art_painting', 'cartoon', 'sketch']

# 데이터 전처리
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
def cross_domain_supcon_loss(features, labels, domains, temperature=0.07):
    """
    features: (B, D)  - normalized feature
    labels:   (B,)
    domains:  (B,)
    """
    B = features.size(0)

    # cosine similarity
    sim = torch.matmul(features, features.T) / temperature  # (B, B)

    # mask: same class
    label_mask = labels.unsqueeze(0) == labels.unsqueeze(1)
    # mask: different domain
    domain_mask = domains.unsqueeze(0) != domains.unsqueeze(1)

    positive_mask = label_mask & domain_mask
    positive_mask.fill_diagonal_(False)

    # log-softmax over rows
    sim_exp = torch.exp(sim)
    sim_exp = sim_exp * (~torch.eye(B, device=features.device).bool())

    log_prob = sim - torch.log(sim_exp.sum(dim=1, keepdim=True) + 1e-8)

    # mean over positives
    num_pos = positive_mask.sum(dim=1)
    loss = -(log_prob * positive_mask).sum(dim=1) / (num_pos + 1e-8)

    # only keep samples that have at least one positiv
    loss = loss[num_pos > 0]

    return loss.mean()

In [ ]:
def freeze_bn_running_stats(model):
    """
    BN의 running mean/var 업데이트를 막기 위해 BN 모듈만 eval로 둔다.
    (gamma/beta는 학습 가능하게 유지)
    """
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.eval()
            if m.weight is not None:
                m.weight.requires_grad = True
            if m.bias is not None:
                m.bias.requires_grad = True

In [ ]:
# 학습 함수
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    freeze_bn_running_stats(model)

    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc='Training')
    for inputs, labels, domains in pbar:
        inputs = inputs.to(device)
        labels = labels.to(device)
        domains = domains.to(device)

        optimizer.zero_grad()

        # wrapper는 (logits, features)를 반환
        logits, features = model(inputs)

        # CE loss
        ce_loss = criterion(logits, labels)

        # cross-domain alignment loss
        features = nn.functional.normalize(features, dim=1)
        align_loss = cross_domain_supcon_loss(
            features, labels, domains
        )

        lambda_align = 0.1
        loss = ce_loss + lambda_align * align_loss

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # sanity check (첫 배치에서만)
        if pbar.n == 0:
            pbar.write(
                f"Domain IDs: {domains.unique().tolist()} | "
                f"CE: {ce_loss.item():.3f} | Align: {align_loss.item():.3f}"
            )

        pbar.set_postfix({
            'loss': f'{running_loss/(pbar.n+1):.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })

    return running_loss / len(loader), 100. * correct / total

In [ ]:
# 평가 함수
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels, _ in tqdm(loader, desc='Evaluating', leave=False):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # wrapper는 (logits, features)를 반환
            logits, _ = model(inputs)

            loss = criterion(logits, labels)

            running_loss += loss.item()
            _, predicted = logits.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return running_loss / len(loader), 100. * correct / total


In [ ]:
# Domain Generalization 실험
def run_domain_generalization(data_root, test_domain):
    """
    Leave-one-domain-out 방식으로 실험 수행
    """
    print(f"\n{'='*80}")
    print(f"Testing on domain: {test_domain}")
    print(f"Training on domains: {[d for d in all_domains if d != test_domain]}")
    print(f"{'='*80}\n")

    # -------------------------------------------------
    # 1. 학습 / 테스트 도메인 분리
    # -------------------------------------------------
    train_domains = [d for d in all_domains if d != test_domain]

    train_dataset = PACSDataset(
        data_root, train_domains, transform=transform_train
    )
    test_dataset = PACSDataset(
        data_root, [test_domain], transform=transform_test
    )

    if len(train_dataset) == 0 or len(test_dataset) == 0:
        print("Error: No samples found. Check dataset paths.")
        return None, None

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        drop_last=True,
        worker_init_fn=lambda worker_id: np.random.seed(42 + worker_id)
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # -------------------------------------------------
    # 2. 모델 생성 (Wrapper 포함)
    # -------------------------------------------------
    backbone = create_model(
        'mobilenetv4_conv_small.e2400_r224_in1k',
        pretrained=True,
        num_classes=num_classes
    )

    model = MobileNetV4WithFeature(backbone)
    model = model.to(device)

    # BN running stats 고정
    freeze_bn_running_stats(model)

    # -------------------------------------------------
    # 3. 손실 / 옵티마이저 / 스케줄러
    # -------------------------------------------------
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs
    )

    # -------------------------------------------------
    # 4. 학습 루프
    # -------------------------------------------------
    best_acc = 0.0
    results = []

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')
        print('-' * 60)

        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, device
        )
        test_loss, test_acc = evaluate(
            model, test_loader, criterion, device
        )

        scheduler.step()

        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Test  Loss: {test_loss:.4f}, Test  Acc: {test_acc:.2f}%')

        results.append({
            'epoch': epoch + 1,
            'train_acc': train_acc,
            'test_acc': test_acc
        })

        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(
                model.state_dict(),
                f'best_model_{test_domain}.pth'
            )
            print(f'✓ Best model saved! Accuracy: {best_acc:.2f}%')

    return best_acc, results


In [ ]:
# 메인 실험 실행
if __name__ == "__main__":
    # PACS 데이터셋 Kaggle에서 다운로드
    try:
        data_root = download_pacs()
        print(f"\nDataset loaded from: {data_root}")
    except Exception as e:
        print(f"Failed to download dataset: {e}")
        print("Please run: pip install kagglehub")
        exit(1)

    print("\nPACS Domain Generalization Experiment")
    print("="*80)
    print(f"Using dataset from: {data_root}")
    print("="*80)

    # 모든 도메인에 대해 실험 수행
    all_results = {}

    for test_domain in all_domains:
        best_acc, results = run_domain_generalization(data_root, test_domain)
        if best_acc is not None:
            all_results[test_domain] = best_acc

    # 최종 결과 출력
    print("\n" + "="*80)
    print("Final Results - Domain Generalization Performance")
    print("="*80)

    if all_results:
        for domain, acc in all_results.items():
            print(f"{domain:15s}: {acc:.2f}%")

        avg_acc = sum(all_results.values()) / len(all_results)
        print("-"*80)
        print(f"{'Average':15s}: {avg_acc:.2f}%")
        print("="*80)
    else:
        print("No results available. Please check your data path.")
        print("\nTroubleshooting:")
        print("1. Make sure kagglehub is installed: pip install kagglehub")
        print("2. Check if Kaggle authentication is set up properly")
        print("3. Verify dataset path structure")

Using Colab cache for faster access to the 'pacs-dataset' dataset.
Path to dataset files: /kaggle/input/pacs-dataset

Dataset loaded from: /kaggle/input/pacs-dataset

PACS Domain Generalization Experiment
Using dataset from: /kaggle/input/pacs-dataset

Testing on domain: photo
Training on domains: ['art_painting', 'cartoon', 'sketch']

Found domain 'art_painting' at: /kaggle/input/pacs-dataset/dct2_images/dct2_images/art_painting
Found domain 'cartoon' at: /kaggle/input/pacs-dataset/dct2_images/dct2_images/cartoon
Found domain 'sketch' at: /kaggle/input/pacs-dataset/dct2_images/dct2_images/sketch
Loaded 8321 samples from domains: ['art_painting', 'cartoon', 'sketch']
Found domain 'photo' at: /kaggle/input/pacs-dataset/dct2_images/dct2_images/photo
Loaded 1670 samples from domains: ['photo']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/15.2M [00:00<?, ?B/s]


Epoch 1/30
------------------------------------------------------------


Training:   2%|▏         | 5/260 [00:03<01:58,  2.14it/s, loss=4.9084, acc=9.77%]

...

Train Loss: 1.5002, Train Acc: 55.18%
Test  Loss: 1.6804, Test  Acc: 37.67%

Epoch 30/30
------------------------------------------------------------


Training:   1%|          | 1/189 [00:00<01:30,  2.07it/s, loss=3.3487, acc=50.00%]

Domain IDs: [0, 1, 2] | CE: 1.353 | Align: 3.483


Training: 100%|██████████| 189/189 [00:15<00:00, 12.21it/s, loss=1.4933, acc=55.52%]
                                                             

Train Loss: 1.4854, Train Acc: 55.52%
Test  Loss: 1.6875, Test  Acc: 37.54%

Final Results - Domain Generalization Performance
photo          : 37.13%
art_painting   : 28.71%
cartoon        : 32.08%
sketch         : 38.66%
--------------------------------------------------------------------------------
Average        : 34.14%
